This notebook is meant for registering between the 24 polarization state combinations in Mueller registration using the instrument at the Vitkin laboratory in Toronto, Canada.  

Each time the instrument shifts polarization state, the image shifts - either greatly for output state, or slightly for input state.  This code implements pre-calculated transforms to correct for these shifts and then to print the resulting images to 24 tifs.

In [1]:
import SimpleITK as sitk
import javabridge
import bioformats as bf
from pathlib import Path
import os

We first need to start up the javabridge, which lets us open up CZI files. 

In [2]:
javabridge.start_vm(class_path=bf.JARS, max_heap_size='8G')

Next we declare the paths to the directory holding the image files, the path to the transform directory, and the path to the output directory

In [3]:
mhr_dir = Path(r'F:\Research\Polarimetry\Data 01 - Raw and imageJ proccessed images\Mueller raw\MHR czi images')
mhr_transform_dir = Path(r'F:\Research\Polarimetry\Data 01 - Raw and imageJ proccessed images\Mueller raw\MHR Transforms')
mhr_output_dir = Path(r'F:\Research\Polarimetry\Data 01 - Raw and imageJ proccessed images\Mueller raw\MHR Registered')


Now we declare some necessary variables - first, the resolution of the czi images we will be processing, and the prefix for how the transforms are named.

In [4]:
mhr_transform_prefix = 'MHR_Position'
mhr_resolution = 0.81

Now we define a few functions that will let us perform this registration automatically.

In [7]:
def list_filetype_in_dir(file_dir, file_ext):
        """Given a directory path, return all files of given file type as a list"""
        return [Path(file_dir, f) for f in os.listdir(file_dir) if f.endswith(file_ext)]

    
def apply_transform(fixed_image: sitk.Image, moving_image: sitk.Image, transform_path):
        transform = sitk.ReadTransform(str(transform_path))
        registered_image = sitk.Resample(moving_image, fixed_image, transform,
                                         sitk.sitkLinear, 0.0, moving_image.GetPixelID())
        
        return registered_image
    

def czi_timepoint_to_sitk_image(path_file, position, resolution, resolution_unit='microns'):
        """
        Open a timepoint from a czi image and make it into an ITK image.  Warning: requires a running javabridge.
        
        :param path_file: Path to the czi file
        :param position: Timepoint to open
        :param resolution: Resolution of the czi image
        :param resolution_unit: The unit of measure (e.g. microns) of the czi image
        :return: A SimpleITK image made from the timepoint
        """
        array = bf.load_image(str(path_file), t=position)
        image = sitk.GetImageFromArray(array)
        image.SetSpacing([resolution, resolution])
        image.SetMetaData('Unit', resolution_unit)
        
        return image
    

def apply_polarization_transforms(path_image, output_dir, transform_dir, transform_prefix, resolution,
                                  skip_existing_images=True):
        """
        Apply pre-calculated transforms onto a single mueller polarimetry image

        :param path_image: path to the image being processed
        :param output_dir: directory to save the image to
        :param resolution: resolution of the image file
        :return:
        """
        print('Applying transforms to {0}'.format(path_image.stem))
        
        fixed_image = czi_timepoint_to_sitk_image(path_image, 0, resolution)

        for num in range(24):
                output_path = Path(output_dir, path_image.stem + '_' + str(num + 1) + '.tif')
                if skip_existing_images and output_path.is_file():
                        continue

                moving_image = czi_timepoint_to_sitk_image(path_image, num, resolution)
                transform_path = Path(transform_dir, transform_prefix + '_' + str(num + 1) + '.tfm')
                
                if num == 0:
                        sitk.WriteImage(fixed_image, str(output_path))
                else:
                        registered_image = apply_transform(fixed_image, moving_image, str(transform_path))
                        sitk.WriteImage(registered_image, str(output_path))


def bulk_apply_polarization_transforms(dir_input, dir_output, transform_dir, transform_prefix,
                                       resolution, skip_existing_images=True):
        """
        Apply pre-calculated transforms onto a whole directory of mueller polarimetry images

        :param dir_input: Directory holding both images and the transforms.csv file
        :param dir_output: Directory to write resulting images to
        :param resolution: Resolution of the image files
        :param skip_existing_images: Whether to skip applying the transform if files already exist
        :return:
        """
        file_list = list_filetype_in_dir(dir_input, 'czi')
        for file in file_list:
                dir_output_file = Path(dir_output, file.stem)
                os.makedirs(dir_output_file, exist_ok=True)
                
                apply_polarization_transforms(file, dir_output_file, transform_dir, transform_prefix, resolution,
                                              skip_existing_images=skip_existing_images)

Finally, we just run the bulk apply transforms function to apply them to every czi image in the directory.

In [6]:
bulk_apply_polarization_transforms(mhr_dir, mhr_output_dir, mhr_transform_dir, mhr_transform_prefix, mhr_resolution)

Applying transforms to 1045- slide 1
Applying transforms to WP2


Once the transforms are complete, we end the javabridge to free up the process.

In [8]:
javabridge.kill_vm()
